<a href="https://colab.research.google.com/github/yammochi/bahamuta/blob/main/gemini_%2Brag_%E6%9E%B6%E6%A7%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前置作業GOOGLE_API_KEY = gemini api的金鑰

# 請到這裡申請 https://aistudio.google.com/app/apikey




## 一.工具

In [1]:
!pip install -q -U google-generativeai
!pip install faiss-cpu
!pip install sentence-transformers
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

## 二.創造RAG向量資料庫 請依照以下格式填入或是直接上傳符合格式的 以下只是示範 實測大概要50組左右 如果要放劇情請差不多每6~7段一組 像是"title": "劇情1", "title": "劇情2"....... 不然太長

In [ ]:
%%writefile medical_knowledge.json
data = [
    {"title": "全名", "content": "長崎そよ"},
    {"title": "學校", "content": "月之森女子學園"},
    {"title": "名言", "content": "為什麼要演奏春日影!!!!!"},
]

Writing medical_knowledge.json


## 三.我們採用intfloat的Embedding模型 對於多國語言來說非常適合跟faiss的向量檢索庫 如果對token數不滿意可以參考這個評比網站 https://ihower.tw/blog/archives/12167

In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import json

# embedding模型
model = SentenceTransformer('intfloat/multilingual-e5-large')

# 資料集
with open('medical_knowledge.json', 'r') as f:
    data = json.load(f)

# 計算向量
texts = [item['content'] for item in data]
embeddings = model.encode(texts)

# 建立向量資料庫
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# 儲存檢索庫
faiss.write_index(index, 'medical_index.faiss')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

## 四.開啟gradio介面

In [ ]:
import os
import json
import google.generativeai as genai
from google.colab import userdata
from sentence_transformers import SentenceTransformer
import faiss
import gradio as gr

# 設定 Google API 金鑰
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

# 載入檢索庫和embedding模型
model = SentenceTransformer('intfloat/multilingual-e5-large')
index = faiss.read_index('medical_index.faiss')

# 載入資料集
with open('medical_knowledge.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 定義檢索函數
def retrieve(query, k=3):
    query_vec = model.encode([query])
    distances, indices = index.search(query_vec, k)
    return indices[0]  # 返回最相關的段落索引

# 對話歷史檔案路徑
conversation_history_file = '/content/conversation_history.txt'

# 載入對話歷史函數
def load_conversation_history():
    try:
        with open(conversation_history_file, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return ""

# 儲存對話歷史函數
def save_conversation_history(history):
    with open(conversation_history_file, 'w', encoding='utf-8') as f:
        f.write(history)

# 問答函數
def answer_question(question):
    # 載入對話歷史
    full_conversation_history = load_conversation_history()

    # 檢索相關段落
    indices = retrieve(question)
    context = " ".join([data[i]['content'] for i in indices])

    # 組合包含完整對話歷史的提示
    prompt = f"你是一名叫做爽世的高中生, 在樂團中是個貝斯手 個性端莊淑女。請基於以下上下文和之前的對話歷史回答問題。\n\n對話歷史：\n{full_conversation_history}\n\n上下文：\n{context}\n\n問題：\n{question}"

    # 使用 Gemini 生成回答
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content([prompt])
    answer = response.text

    # 更新完整對話歷史
    full_conversation_history += f"使用者：{question}\n爽世：{answer}\n"

    # 儲存對話歷史
    save_conversation_history(full_conversation_history)

    return answer

# 追加資料到 medical_knowledge.json
def append_data(title, content):
    new_data = {"title": title, "content": content}
    with open('medical_knowledge.json', 'r+', encoding='utf-8') as f:
        try:
            existing_data = json.load(f)
        except json.JSONDecodeError: # 處理空檔案
            existing_data = []
        existing_data.append(new_data)
        f.seek(0)  # 回到檔案開頭
        json.dump(existing_data, f, indent=2, ensure_ascii=False)
        f.truncate() # 清除後續內容
        global data #更新全域變數data
        data = existing_data
        # 重新建立索引
        embeddings = model.encode([item['content'] for item in data])
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index.add(embeddings)
        faiss.write_index(index, "medical_index.faiss")


    return "資料已成功新增！"


# 建立 Gradio 介面
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="輸入你的問題:")
    clear = gr.Button("清除歷史紀錄")

    with gr.Row():
      title_input = gr.Textbox(label="標題")
      content_input = gr.Textbox(label="內容")
      append_button = gr.Button("新增資料 很慢 差不多一次60秒")

    def user(user_message, history):
        response = answer_question(user_message)
        history.append((user_message, response))
        return "", history

    def clear_history():
        save_conversation_history("")  # 清空檔案內容
        return []

    msg.submit(user, [msg, chatbot], [msg, chatbot])
    clear.click(clear_history, outputs=chatbot)
    append_button.click(append_data, inputs=[title_input, content_input], outputs=gr.Textbox(label="狀態"))

demo.launch(debug=True)




/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a890fe3ee83cee53ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
